## 미니배치

In [78]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test)=load_mnist(normalize=True, one_hot_label=True)

#60000개의 이미지가 있음
print(x_train.shape) #28*28이 faltten
print(t_train.shape) #one-hot encoding

(60000, 784)
(60000, 10)


In [82]:
train_size=x_train.shape[0]
batch_size=10
batch_mask=np.random.choice(train_size, batch_size)
x_batch=x_train[batch_mask]
y_batch=t_train[batch_mask]

In [90]:
x_batch.shape

(10, 784)

## 손실함수 : (배치용)교차 엔트로피 오차 구현하기 
벡터일 때 : ndim : 차원, size : 원소의 개수  
행렬일 경우 ndim=2임

In [ ]:
def cross_entrophy_error(y,t):
    if y.ndim==1:
        t=t.reshape(1, t.size) #벡터를 행렬로 만듦
        y=y.reshape(1, y.size) #벡터를 행렬로 만듦
        
    batch_size=y.shape[0] #shape[0] : 행의 개수, shape[1] : 열의 개수
    return -np.sum(t*np.log(y+1e-7)) / batch_size

In [97]:
t=t_train[0]
t=y.reshape(1, y.size)
t.shape

(1, 10)

정답 레이블이 one-hot encoding이 아니라 "2"나 "7" 등의 숫자 레이블로 주어졌을 때의 교차 엔트로피 오차

In [108]:
def cross_entrophy_error(y,t):
    if y.ndim==1:
        t=t.reshape(1, t.size) #벡터를 행렬로 만듦
        y=y.reshape(1, y.size) #벡터를 행렬로 만듦
        
    batch_size=y.shape[0] #shape[0] : 행의 개수, shape[1] : 열의 개수
    return -np.sum(np.log(y[np.arrange(batch_size),t]+1e-7)) / batch_size #각 데이터의 정답 레이블에 해당하는 신경망의 출력을 추출

In [109]:
y=np.array([[0,1,2,3,4,5,6,7,8,9],
           [1,2,3,4,5,6,7,8,9,0],
           [2,3,4,5,6,7,8,9,0,1]])
y[np.arange(2), [2,7]]

array([2, 8])

## 기울기
도대체 왜 답이 안 나오는지 모르겠다...코드는 맞는 거 같은데..

In [31]:
import numpy as np

In [37]:
def numerical_gradient(f,x):
    h=1e-3
    grad=np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val=x[idx]
        x[idx]=tmp_val+h
        fxh1=f(x)
        
        x[idx]=tmp_val-h
        fxh2=f(x)
        
        grad[idx]=(fxh1-fxh2)/(2*h)
        x[idx]=tmp_val
        
    return grad

In [38]:
def function_2(x):
    return x[0]**2+x[1]**2

In [44]:
numerical_gradient(function_2, np.array([3.0, 4.0]))

array([6., 8.])

In [45]:
numerical_gradient(function_2, np.array([0.0,2.0]))

array([0., 4.])

In [46]:
numerical_gradient(function_2, np.array([3.0,0.0]))

array([6., 0.])

## 경사하강법

In [49]:
#f : 최적화하려는 함수, init_x : 초기값, lr: 학습률, step_num : 경사법에 따른 반복 횟수
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x=init_x
    
    for i in range(step_num):
        grad=numerical_gradient(f,x)
        x-=lr*grad #기울기 업데이트
    return x

In [50]:
init_x=np.array([-3.0, 4.0])

In [51]:
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100) #실제 최소값인 원점에 가까워짐

array([-6.11110793e-10,  8.14814391e-10])

## 신경망에서의 기울기

In [67]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

In [68]:
class simpleNet:
    def __init__(self):
        self.W=np.random.randn(2,3) #정규분포 초기화, 표본의 개수는 2개이며 node는 3개(편향이 없다고 생각하자.)
    
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z=self.predict(x)
        y=softmax(z)
        loss=cross_entropy_error(y,t)
        return loss

In [69]:
net=simpleNet()
print(net.W)

[[ 1.10281804  1.60317482  1.48500379]
 [ 0.64682722  0.05600907 -0.88921157]]


In [70]:
x=np.array([0.6, 0.9]) 
p=net.predict(x)
print(p)

[1.24383533 1.01231305 0.09071186]


In [71]:
t=np.array([0,0,1])
net.loss(x,t)

1.8993245311826772

주의해야 할 것 : 아래에서 사용하는 numerical gradient는 위의 작성한 gradient_numeric 코드로 사용불가 그 이유는 2d만 포괄할 수 있기 때문에
따라서 common.gradient의 numerical_gradient 함수를 불러온 것임

In [73]:
# common.gradient에 있는 함수
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad


In [74]:
def f(W):
    return net.loss(x,t)
dW=numerical_gradient(f, net.W)

In [75]:
dW

array([[ 0.28449829,  0.22569962, -0.51019791],
       [ 0.42674744,  0.33854942, -0.76529686]])

## 신경망 학습의 절차
1단계-미니배치  
훈련 데이터 중 일부를 무작위로 가져온다.  
  
2단계-기울기 산출  
미니배치의 손실함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다.  
기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다.  
  
3단계-매개변수 갱신  
가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.  
  
4단계-반복
1~3단계를 반복합니다.

In [112]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    #변수선언
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        
        self.params={}
        self.params['W1']=weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1']=np.zeros(hidden_size)
        self.params['W2']=weight_init_std*np.random.randn(hidden_size, ouput_size)
        self.params['b2']=np.zeros(output_size)
    
    #forward
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1=np.dot(x, W1)+b1
        z1=sigmoid(a1)
        a2=np.dot(z1, W2)+b2
        y=softmax(a2)
        
        return y
    
    #손실함수
    # x:입력 데이터, t:정답레이블
    def loss(self, x, t):
        y=self.predict(x)
        return cross_entropy_error(y,t)
    
    def accuracy(self, x, t):
        y=self.predict(x)
        y=np.argmax(y, axis=1)
        t=np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) /float(x.shape[0])
        return accuracy
    
    #경사하강법 시행
    def numercial_gradient(self, x, t):
        loss_W=lambda W: self.loss(x, t)
        
        grads={}
        grads['W1']=numerical_gradient(loss_W, self.params['W1'])
        grads['b1']=numerical_gradient(loss_W, self.params['b1'])
        grads['W2']=numerical_gradient(loss_W, self.params['W2'])
        grads['b2']=numerical_gradient(loss_W, self.params['b2'])
        
        return grads

## TwoLayerNet class와 MNIST 데이터셋을 사용해 학습

앞 코드에서 미니배치 코드를 추가

In [113]:
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

In [115]:
(x_train, t_train), (x_test, t_test)=load_mnist(normalize=True, one_hot_label=True)

train_loss_list=[]
train_acc_list=[]
test_acc_list=[]

#1에폭당 반복 수
iter_per_epoch=max(train_size/batch_size, 1)

#hyperparameter
iters_num=10000 #반복횟수
train_size=x_train.shape[0]
batch_size=100 #미니배치의 크기
learning_rate=0.1

In [116]:
network=TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    #미니배치 획득
    batch_mask=np.random.choice(train_size, batch_size)
    x_batch=x_train[batch_mask]
    t_batch=t_train[batch_mask]
    
    #기울기 계산
    grad=network.numerical_gradient(x_batch, t_batch)
    #grad=network.gradient(x_batch, t_batch)는 다음 장에서!
    
    #매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key]-=learning_rate*grad[key]
        
    #학습 경과 기록
    loss=network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    
    #1에폭당 정확도 계산
    if i % iter_per_epoch==0:
        train_acc=network.accuracy(x_train, t_train)
        test_acc=network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc : "+str(train_acc)", "+str(test_acc))
    
    #학습시간이 걸리므로 코드이해만 하고 넘어가기
    

KeyboardInterrupt: 